# Crawling des österreichischen Volleyballteams - Kader Damen

## Einführung
Im 1. Assignment werden wir Informationen über die Volleyballspielerinnen des österreichischen Nationalteams sammeln. Es existiert eine Liste des Kaders sowie Detailseiten zu jeder Person, in denen die folgenden Daten enthalten sind:
- Name
- Körpergröße
- Dressnummer
- Position
  
Als Ergebnis präsentieren wir ein CSV-File mit den o.g. Datenpunkten und Informationen zu jeder Spielerin des Kaders.

Die Umsetzung erfolgt, wie im Assignment vorgegeben, mithilfe der Python-Libraries <i>Selenium</i> und <i>Beautiful Soup 4</i>.

## Arbeitsaufteilung
**2. Selenium-Crawler auf die Haupt- und Damenkaderseite:** Ecker Annina
**3. Laden von Spielerinnen-Daten in einen DataFrame (BeautifulSoup4):** Cesar Laura
**4. Abgleich der DFs und Bereinigen der Daten:** Dilly Julian


### 1. Installation und Importieren der benötigten Libraries

In [1]:
# !pip install selenium beautifulsoup4 requests pandas

In [2]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
import pandas as pd
from bs4 import BeautifulSoup
import requests

### 2. Selenium-Crawler auf die Haupt- und Damenkaderseite

In [3]:
# Chrome im Headless-Modus starten, da sonst Browser-Fenster aufpoppt
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
base_url = "https://www.volleynet.at"

# WebDriver starten
driver = webdriver.Chrome(options=chrome_options)

try:
    driver.get(base_url)
    # Warten, bis die Seite vollständig geladen ist und Definieren von explizitem Wait
    sleep_long = time.sleep(5)
    
    sleep_long
    wait = WebDriverWait(driver, 10)

    # Zuerst das Cookie-Banner schließen
    try:
        cookie_close_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.cmplz-close")))
        cookie_close_button.click()
        print("✅ Cookie-Banner geschlossen")
    except Exception as e:
        print(f"\033[91m ❌ Cookie-Banner nicht gefunden oder konnte nicht geschlossen werden: {e}\033[0m")

    # Warte kurz nach dem Schließen des Cookie-Banners
    sleep_short = time.sleep(2)
    sleep_short

    # Navigiere zum Nationalteams-Menü
    try:
        nationalteams_menu = wait.until(EC.element_to_be_clickable((By.ID, 'menu-item-4424')))
        print(f"✅ Nationalteams Menü gefunden (\033[94m{nationalteams_menu.text}\033[0m)")
        
        action = ActionChains(driver)
        action.move_to_element(nationalteams_menu).perform()
        nationalteams_menu.click()
        sleep_short
        
    except Exception as e:
        print(f"\033[91m ❌ Fehler beim Navigieren zum Nationalteams-Menü: {e}\033[0m")

    # Navigiere zum Damenkader-Menü
    try:
        damen_menu = wait.until(EC.element_to_be_clickable((By.ID, 'menu-item-4434')))
        print(f"✅ Damenkader-Menü gefunden (\033[94m{damen_menu.text}\033[0m)")
        sleep_short
        
    except Exception as e:
        print(f"\033[91m ❌ Fehler beim Finden des Damenkader-Menüs: {e}\033[0m")

    # Selenium soll nun den Kader-Link anklicken
    try:
        kader_link = wait.until(EC.element_to_be_clickable((By.ID, 'menu-item-4471')))
        print(f"✅ Kader-Link gefunden (\033[94m{kader_link.text}\033[0m)")
        
        action.move_to_element(kader_link).perform()
        kader_link.click()

        sleep_long
        print(f"✅ Aktuelle URL nach dem Klick: \033[94m{driver.current_url}\033[0m")
        
    except Exception as e:
        print(f"\033[91m ❌ Fehler beim Finden oder Klicken auf den Kader-Link: {e}\033[0m")

except Exception as e:
    print(f"\033[91m ❌ Ein allgemeiner Fehler ist aufgetreten: {e}\033[0m")

✅ Cookie-Banner geschlossen
✅ Nationalteams Menü gefunden (NATIONALTEAMS)
✅ Damenkader-Menü gefunden (Damen
Informationen
Tickets)
✅ Kader-Link gefunden (Kader)
✅ Aktuelle URL nach dem Klick: https://www.volleynet.at/nationalteams/damen/kader/


### 3. Laden von Spielerinnen-Daten in einen DataFrame (BeautifulSoup4)

In [4]:
try:
    # driver.page source is content of current page
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    
    # Extract player information
    player_table = soup.find(id='DataTables_Table_0')
    players = player_table.find_all('tr', class_=['tablehell', 'tabledunkel'])
    player_data = []
    
    for player in players:
        try:
            dressnumber = player.find_all('td')[0].get_text(strip=True)
            name = player.find_all('td')[1].find('a').get_text(strip=True)
            position = player.find_all('td')[3].get_text(strip=True)
            nationality = player.find_all('td')[2].get_text(strip=True)

            # Scrape player height from profile page
            try:
                link = player.find('a').get('href')
                full_link = base_url + link
                response = requests.get(full_link)
                profile_soup = BeautifulSoup(response.content, 'html.parser')
                profile_table = profile_soup.find('tbody')
                height_row = profile_table.find_all('tr')[2]
                height = height_row.find_all('td')[1].get_text(strip=True).split()[0]
            except AttributeError as e:
                print(f"❌ Error retrieving player information from page {player['detail_URL']}: {e}")
            player_data.append([dressnumber, name, position, nationality, height])
        except Exception as e:
            print(f"❌ Error scraping player information: {e}")

    print("✅ Player data has been extracted")
    
    # Save scraped data to csv file
    with open('player_data.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Dressnumber', 'Name', 'Position', 'Nationality', 'Height'])
        writer.writerows(player_data)
    
    print("✅ Player data has been saved to 'player_data.csv'")
    
except Exception as e:
    print(f"❌ Error finding or clicking on the roster link: {e}")

# Ensure that driver gets quittetd even if exception occurs
finally:
    driver.quit()
    print("✅ Webdriver has been quitted")

✅ Player data has been extracted
✅ Player data has been saved to 'player_data.csv'
✅ Webdriver has been quitted


### 4. Abgleich der DFs und Bereinigen der Daten

# Ressourcen & Source-Docs

## Selenium
1. [WebDriver](https://www.geeksforgeeks.org/selenium-webdriver-commands/)
2. [Implizites und explizites Warten](https://www.geeksforgeeks.org/explicit-waits-in-selenium-python/)
3. [ActionChains](https://www.geeksforgeeks.org/action-chains-in-selenium-python/)
4. [element_to_be_clickable](https://selenium-python.readthedocs.io/api.html#module-selenium.webdriver.support.expected_conditions)
5. [move_to_element](https://www.geeksforgeeks.org/move_to_element-method-action-chains-in-selenium-python/?ref=rp)

## Beautiful Soup 4
1. [Selenium: page_source](https://www.geeksforgeeks.org/page_source-driver-method-selenium-python/)
2. [Beautiful Soup: find_all() & access links](https://beautiful-soup-4.readthedocs.io/en/latest/#making-the-soup:~:text=One%20common%20task%20is%20extracting%20all%20the%20URLs%20found%20within%20a%20page%E2%80%99s%20%3Ca%3E%20tags%3A)
3. [Beautiful Soup: get_text()](https://beautiful-soup-4.readthedocs.io/en/latest/#get-text)
4. [Requests: request.get()](https://docs.python-requests.org/en/latest/user/quickstart/)
5. [Selenium: driver.quit()](https://www.geeksforgeeks.org/how-to-use-close-and-quit-method-in-selenium-python/)

## Cleanup Code
1. a
2. b

# Challenges bei der Implementierung


#### Cesar Laura, Scraping
- Generell war es am anfang etwas verwirrend wie ich mich in den extracted daten navigiere und auf die gewünschte Information zugreife

#### Dilly Julian
-
-

#### Ecker Annina, Implementierung von Selenium:
- Zunächst hatte ich Probleme mit dem WebDriver, da ich nicht die `headless`-Variante kannte. Da ich dann zusätzlich wegen dem Cookie-Banner etwas unsicher war, und ob der WebDriver damit immer gut umgehen würde, habe ich mich auch bei der `headless`-Option dazu entschieden, den Cookie-Banner wegklicken zu lassen.
- Die Navigation zu den Menü-Items war grundsätzlich kein Problem, aber es gab einige Eigenschaften bei `element_to_be_clickable` wie das _implicite_ oder _explicite Wait_, wo ich mich einlesen musste.